# feature select one
## 分期支付

In [1]:
import datetime
import numpy as np
import pandas as pd
import joblib
import warnings
import logging
import os
import gc
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import collections
import re
import copy
import lightgbm as lgb

import utils

from tqdm import tqdm
from dateutil.relativedelta import relativedelta
from joblib import Parallel, delayed
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.inspection import permutation_importance


# pd.set_option('display.max_columns', None)
# pd.set_option('max_row', 500)
warnings.filterwarnings('ignore')
tqdm.pandas(desc='pandas bar')

KeyboardInterrupt: 

In [ ]:
lgb.__version__

## 数据

In [ ]:
# df = pd.read_csv('data/sample_label_feature_fusion_obs_dt_20231217_20240204.txt', sep='\t', encoding='utf-8')
# df = pd.read_csv('data/sample_label_feature_asp1_obs_dt_20231217_20240204.txt', sep='\t', encoding='utf-8')
# df = pd.read_csv('data/sample_label_feature_asp2_obs_dt_20231217_20240204.txt', sep='\t', encoding='utf-8')
# df = pd.read_csv('data/sample_label_feature_bank_card_debit_obs_dt_20231217_20240204.txt', sep='\t', encoding='utf-8')

# df = pd.read_csv('data/sample_label_feature_fusion_instal_obs_dt_20231217_20240303.txt', sep='\t', encoding='utf-8')
# df = pd.read_csv('data/sample_label_feature_asp1_instal_obs_dt_20231217_20240303.txt', sep='\t', encoding='utf-8')
# df = pd.read_csv('data/sample_label_feature_asp2_instal_obs_dt_20231217_20240303.txt', sep='\t', encoding='utf-8')
# df = pd.read_csv('data/sample_label_feature_dbc_instal_obs_dt_20231217_20240303.txt', sep='\t', encoding='utf-8')

print(df.shape)
df.head()

In [ ]:
[x for x in df.columns if x.endswith('.1')]

In [ ]:
df.info()

In [ ]:
df[['pay_type']].info()

In [ ]:
df[['obs_dt', 'uid']].groupby(by=['obs_dt']).count()

In [ ]:
df[['pay_type', 'default_chose_type', 'label', 'uid']].groupby(by=['pay_type', 'default_chose_type', 'label']).count()

In [ ]:
df = df[df['pay_type']==1]
df.reset_index(drop=True, inplace=True)
print(df.shape)
df.head()

In [ ]:
df['obs_dt'] = pd.to_datetime(df['obs_dt'])
df['dt'] = pd.to_datetime(df['dt'])

# utils.save_pickle(df, 'data/df_sample_label_feature_fusion_one_obs_dt_20231217_20240204.pickle')
# utils.save_pickle(df, 'data/df_sample_label_feature_asp1_one_obs_dt_20231217_20240204.pickle')
# utils.save_pickle(df, 'data/df_sample_label_feature_asp2_one_obs_dt_20231217_20240204.pickle')
# utils.save_pickle(df, 'data/df_sample_label_feature_bcd_one_obs_dt_20231217_20240204.pickle')

utils.save_pickle(df, 'data/df_sample_label_feature_fusion_instal_obs_dt_20231217_20240303.pickle')
# utils.save_pickle(df, 'data/df_sample_label_feature_asp1_instal_obs_dt_20231217_20240303.pickle')
# utils.save_pickle(df, 'data/df_sample_label_feature_asp2_instal_obs_dt_20231217_20240303.pickle')
# utils.save_pickle(df, 'data/df_sample_label_feature_dbc_instal_obs_dt_20231217_20240303.pickle')

## 特征选择

* 方差&人工

In [ ]:
# df = utils.load_pickle('data/df_sample_label_feature_fusion_one_obs_dt_20231217_20240204.pickle')
# df = utils.load_pickle('data/df_sample_label_feature_asp1_one_obs_dt_20231217_20240204.pickle')
# df = utils.load_pickle('data/df_sample_label_feature_asp2_one_obs_dt_20231217_20240204.pickle')
# df = utils.load_pickle('data/df_sample_label_feature_bcd_one_obs_dt_20231217_20240204.pickle')

df = utils.load_pickle('data/df_sample_label_feature_dbc_instal_obs_dt_20231217_20240303.pickle')

print(df.shape)
df.head()

In [ ]:
df_des = utils.df_des(df)
print(df_des.shape)
df_des.head()

In [ ]:
# df_des.to_csv('data/df_des_sample_label_feature_fusion_one_obs_dt_20231217_20240204.csv', encoding='utf-8')
# df_des.to_csv('data/df_des_sample_label_feature_asp1_one_obs_dt_20231217_20240204.csv', encoding='utf-8')
# df_des.to_csv('data/df_des_sample_label_feature_asp2_one_obs_dt_20231217_20240204.csv', encoding='utf-8')
# df_des.to_csv('data/df_des_sample_label_feature_bcd_one_obs_dt_20231217_20240204.csv', encoding='utf-8')

df_des.to_csv('data/df_des_sample_label_feature_dbc_instal_obs_dt_20231217_20240303.csv', encoding='utf-8')

In [ ]:
list_feats_id_dt_condition_y = ['uid', 'obs_dt', 'dt', 'pay_type', 'default_chose_type', 'label']
list_feats_x = [x for x in df.columns if x not in list_feats_id_dt_condition_y]
print(len(list_feats_x))
list_feats_x[:10]

In [ ]:
# utils.save_pickle(list_feats_x, 'data/list_feats/list_feats_x_fusion_20231217_20240204.pickle')
# utils.save_pickle(list_feats_x, 'data/list_feats/list_feats_x_asp1_20231217_20240204.pickle')
# utils.save_pickle(list_feats_x, 'data/list_feats/list_feats_x_asp2_20231217_20240204.pickle')
# utils.save_pickle(list_feats_x, 'data/list_feats/list_feats_x_bcd_20231217_20240204.pickle')

utils.save_pickle(list_feats_x, 'data/list_feats/dbc/list_feats_x_dbc_20231217_20240303.pickle')

In [ ]:
list_feats_x_std_0 = [x for x in df_des[df_des['std']==0].index if x not in ['pay_type']]
print(len(list_feats_x_std_0))
list_feats_x_std_0[:10]

In [ ]:
# utils.save_pickle(list_feats_x_std_0, 'data/list_feats/list_feats_x_std_0_fusion_20231217_20240204.pickle')
# utils.save_pickle(list_feats_x_std_0, 'data/list_feats/list_feats_x_std_0_asp1_20231217_20240204.pickle')
# utils.save_pickle(list_feats_x_std_0, 'data/list_feats/list_feats_x_std_0_asp2_20231217_20240204.pickle')
# utils.save_pickle(list_feats_x_std_0, 'data/list_feats/list_feats_x_std_0_bcd_20231217_20240204.pickle')

utils.save_pickle(list_feats_x_std_0, 'data/list_feats/dbc/list_feats_x_std_0_dbc_20231217_20240303.pickle')

In [ ]:
list_feats_x_std = [x for x in list_feats_x if x not in list_feats_x_std_0]
print(len(list_feats_x_std))
list_feats_x_std[:10]

In [ ]:
# utils.save_pickle(list_feats_x_std, 'data/list_feats/list_feats_x_std_fusion_20231217_20240204.pickle')
# utils.save_pickle(list_feats_x_std, 'data/list_feats/list_feats_x_std_asp1_20231217_20240204.pickle')
# utils.save_pickle(list_feats_x_std, 'data/list_feats/list_feats_x_std_asp2_20231217_20240204.pickle')
# utils.save_pickle(list_feats_x_std, 'data/list_feats/list_feats_x_std_bcd_20231217_20240204.pickle')

utils.save_pickle(list_feats_x_std, 'data/list_feats/dbc/list_feats_x_std_dbc_20231217_20240303.pickle')

* 单因素方差分析（假设检验，f检验）

In [ ]:
# df = utils.load_pickle('data/df_sample_label_feature_fusion_one_obs_dt_20231217_20240204.pickle')
# df = utils.load_pickle('data/df_sample_label_feature_asp1_one_obs_dt_20231217_20240204.pickle')
# df = utils.load_pickle('data/df_sample_label_feature_asp2_one_obs_dt_20231217_20240204.pickle')
# df = utils.load_pickle('data/df_sample_label_feature_bcd_one_obs_dt_20231217_20240204.pickle')

df = utils.load_pickle('data/df_sample_label_feature_dbc_instal_obs_dt_20231217_20240303.pickle')

print(df.shape)
df.head()

In [ ]:
df[['pay_type', 'default_chose_type', 'label', 'uid']].groupby(by=['pay_type', 'default_chose_type', 'label']).count()

In [ ]:
# list_feats_x_std = utils.load_pickle('data/list_feats/list_feats_x_std_fusion_20231217_20240204.pickle')
# list_feats_x_std = utils.load_pickle('data/list_feats/list_feats_x_std_asp1_20231217_20240204.pickle')
# list_feats_x_std = utils.load_pickle('data/list_feats/list_feats_x_std_asp2_20231217_20240204.pickle')
# list_feats_x_std = utils.load_pickle('data/list_feats/list_feats_x_std_bcd_20231217_20240204.pickle')

list_feats_x_std = utils.load_pickle('data/list_feats/dbc/list_feats_x_std_dbc_20231217_20240303.pickle')

print(len(list_feats_x_std))
list_feats_x_std[:10]

In [ ]:
list_var_f = []

try:
    with tqdm(list_feats_x_std) as t:
        for feat in t:
            var_temp = utils.hypothesis_testing(df[[feat, 'label']], [feat], 'label', method='f_classif')
            list_var_f.append(var_temp)
except KeyboardInterrupt:
    t.close()
    raise
t.close()

var_f = pd.concat(list_var_f, axis=0)
var_f.reset_index(drop=True, inplace=True)
var_f.sort_values(by=['score', 'P_value'], ascending=[False, False], inplace=True)
var_f.head()

In [ ]:
list_feats_x_ht_no_select = var_f[var_f['is_selected_PValue']==0]['feature'].values.tolist()
print(len(list_feats_x_ht_no_select))
list_feats_x_ht_no_select[:10]

In [ ]:
# utils.save_pickle(list_feats_x_ht_no_select, 'data/list_feats/list_feats_x_ht_no_select_fusion_20231217_20240204.pickle')
# utils.save_pickle(list_feats_x_ht_no_select, 'data/list_feats/list_feats_x_ht_no_select_asp1_20231217_20240204.pickle')
# utils.save_pickle(list_feats_x_ht_no_select, 'data/list_feats/list_feats_x_ht_no_select_asp2_20231217_20240204.pickle')
# utils.save_pickle(list_feats_x_ht_no_select, 'data/list_feats/list_feats_x_ht_no_select_bcd_20231217_20240204.pickle')

utils.save_pickle(list_feats_x_ht_no_select, 'data/list_feats/dbc/list_feats_x_ht_no_select_dbc_20231217_20240303.pickle')

In [ ]:
# list_feats_x_ht_no_select = utils.load_pickle('data/list_feats/list_feats_x_ht_no_select_fusion_20231217_20240204.pickle')
# list_feats_x_ht_no_select = utils.load_pickle('data/list_feats/list_feats_x_ht_no_select_asp1_20231217_20240204.pickle')
# list_feats_x_ht_no_select = utils.load_pickle('data/list_feats/list_feats_x_ht_no_select_asp2_20231217_20240204.pickle')
# list_feats_x_ht_no_select = utils.load_pickle('data/list_feats/list_feats_x_ht_no_select_bcd_20231217_20240204.pickle')

list_feats_x_ht_no_select = utils.load_pickle('data/list_feats/dbc/list_feats_x_ht_no_select_dbc_20231217_20240303.pickle')

print(len(list_feats_x_ht_no_select))
list_feats_x_ht_no_select[:10]

In [ ]:
list_feats_x_ht = [x for x in list_feats_x_std if x not in list_feats_x_ht_no_select]
print(len(list_feats_x_ht))
list_feats_x_ht[:10]

In [ ]:
# utils.save_pickle(list_feats_x_ht, 'data/list_feats/list_feats_x_ht_fusion_20231217_20240204.pickle')
# utils.save_pickle(list_feats_x_ht, 'data/list_feats/list_feats_x_ht_asp1_20231217_20240204.pickle')
# utils.save_pickle(list_feats_x_ht, 'data/list_feats/list_feats_x_ht_asp2_20231217_20240204.pickle')
# utils.save_pickle(list_feats_x_ht, 'data/list_feats/list_feats_x_ht_bcd_20231217_20240204.pickle')

utils.save_pickle(list_feats_x_ht, 'data/list_feats/dbc/list_feats_x_ht_dbc_20231217_20240303.pickle')

* 模型
* 特征重要性
* PI方法，使用lgb进行建模预测

In [ ]:
# df = utils.load_pickle('data/df_sample_label_feature_fusion_one_obs_dt_20231217_20240204.pickle')

# print(df.shape)

# df_asp1 = utils.load_pickle('data/df_sample_label_feature_asp1_one_obs_dt_20231217_20240204.pickle')
# df_asp2 = utils.load_pickle('data/df_sample_label_feature_asp2_one_obs_dt_20231217_20240204.pickle')
# df_bcd = utils.load_pickle('data/df_sample_label_feature_bcd_one_obs_dt_20231217_20240204.pickle')

df_asp1 = utils.load_pickle('data/df_sample_label_feature_asp1_instal_obs_dt_20231217_20240303.pickle')
df_asp2 = utils.load_pickle('data/df_sample_label_feature_asp2_instal_obs_dt_20231217_20240303.pickle')
df_dbc = utils.load_pickle('data/df_sample_label_feature_dbc_instal_obs_dt_20231217_20240303.pickle')

print(df_asp1.shape)
print(df_asp2.shape)
print(df_dbc.shape)

In [ ]:
df_asp1_rm = pd.read_csv('data/pi/dbc/asp1/cm_credit_card.txt', sep='\t', encoding='utf-8')
print(df_asp1_rm.shape)
df_asp1_rm.head()

In [ ]:
list_feats_id_dt_condition_y = ['uid', 'obs_dt', 'dt', 'pay_type', 'default_chose_type', 'label']

# list_feats_x_ht = utils.load_pickle('data/list_feats/list_feats_x_ht_fusion_20231217_20240204.pickle')

# list_feats_x_ht_1 = utils.load_pickle('data/list_feats/list_feats_x_ht_asp1_20231217_20240204.pickle')
list_feats_x_ht_2 = utils.load_pickle('data/list_feats/list_feats_x_ht_asp2_20231217_20240204.pickle')
# list_feats_x_ht_3 = utils.load_pickle('data/list_feats/list_feats_x_ht_bcd_20231217_20240204.pickle')

list_feats_x_ht_3 = utils.load_pickle('data/list_feats/dbc/list_feats_x_ht_dbc_20231217_20240303.pickle')

list_feats_x_ht_1 = [
    x for x in utils.load_pickle('data/list_feats/list_feats_x_ht_asp1_20231217_20240204.pickle')
    if x not in list(df_asp1_rm['feats'])
]

print(len(list_feats_x_ht_1))
print(len(list_feats_x_ht_2))
print(len(list_feats_x_ht_3))

In [ ]:
%%time
# df = df_asp1[list_feats_id_dt_condition_y+list_feats_x_ht_1].\
#     merge(df_asp2[list_feats_id_dt_condition_y+list_feats_x_ht_2], on=list_feats_id_dt_condition_y, how='left').\
#     merge(df_bcd[list_feats_id_dt_condition_y+list_feats_x_ht_3], on=list_feats_id_dt_condition_y, how='left')

df = df_asp1[list_feats_id_dt_condition_y+list_feats_x_ht_1].\
    merge(df_asp2[list_feats_id_dt_condition_y+list_feats_x_ht_2], on=list_feats_id_dt_condition_y, how='left').\
    merge(df_dbc[list_feats_id_dt_condition_y+list_feats_x_ht_3], on=list_feats_id_dt_condition_y, how='left')

print(df.shape)
df.head()

In [ ]:
[x for x in df.columns if x.endswith('_x')]

In [ ]:
# utils.save_pickle(df, 'data/pi/df_ht_asp_bcd_20231217_20240204.pickle')

# utils.save_pickle(df, 'data/pi/dbc/df_ht_asp_dbc_20231217_20240303.pickle')
utils.save_pickle(df, 'data/pi/dbc/asp1/df_ht_asp_dbc_asp1_20231217_20240303.pickle')

In [ ]:
# df = utils.load_pickle('data/pi/df_ht_asp_bcd_20231217_20240204.pickle')

# df = utils.load_pickle('data/pi/dbc/df_ht_asp_dbc_20231217_20240303.pickle')

df = utils.load_pickle('data/pi/dbc/asp1/df_ht_asp_dbc_asp1_20231217_20240303.pickle')

print(df.shape)
df.head()

In [ ]:
# list_feats_x_ht_1 = utils.load_pickle('data/list_feats/list_feats_x_ht_asp1_20231217_20240204.pickle')
list_feats_x_ht_2 = utils.load_pickle('data/list_feats/list_feats_x_ht_asp2_20231217_20240204.pickle')
# list_feats_x_ht_3 = utils.load_pickle('data/list_feats/list_feats_x_ht_bcd_20231217_20240204.pickle')

list_feats_x_ht_3 = utils.load_pickle('data/list_feats/dbc/list_feats_x_ht_dbc_20231217_20240303.pickle')

list_feats_x_ht_1 = [
    x for x in utils.load_pickle('data/list_feats/list_feats_x_ht_asp1_20231217_20240204.pickle')
    if x not in list(df_asp1_rm['feats'])
]

list_feats_x_ht = list_feats_x_ht_1 + list_feats_x_ht_2 + list_feats_x_ht_3

print(len(list_feats_x_ht_1))
print(len(list_feats_x_ht_2))
print(len(list_feats_x_ht_3))

print(len(list_feats_x_ht))

In [ ]:
# PI
df_y = df['label']
df_X = df[list_feats_x_ht]

print(df_y.shape)
print(df_X.shape)

In [ ]:
%xdel df
%xdel df_asp1
%xdel df_asp2
%xdel df_dbc

In [ ]:
df_X_train, df_X_test, df_y_train, df_y_test = \
    train_test_split(df_X, df_y, test_size=0.2, random_state=2024)
print(df_X_train.shape)
print(df_X_test.shape)
print(df_y_train.shape)
print(df_y_test.shape)

In [ ]:
# utils.save_pickle(df_X_train, 'data/pi/df_ht_X_train_fusion_20231217_20240204.pickle')
# utils.save_pickle(df_X_test, 'data/pi/df_ht_X_test_fusion_20231217_20240204.pickle')
# utils.save_pickle(df_y_train, 'data/pi/df_ht_y_train_fusion_20231217_20240204.pickle')
# utils.save_pickle(df_y_test, 'data/pi/df_ht_y_test_fusion_20231217_20240204.pickle')

# utils.save_pickle(df_X_train, 'data/pi/df_ht_X_train_asp_bcd_20231217_20240204.pickle')
# utils.save_pickle(df_X_test, 'data/pi/df_ht_X_test_asp_bcd_20231217_20240204.pickle')
# utils.save_pickle(df_y_train, 'data/pi/df_ht_y_train_asp_bcd_20231217_20240204.pickle')
# utils.save_pickle(df_y_test, 'data/pi/df_ht_y_test_asp_bcd_20231217_20240204.pickle')

# utils.save_pickle(df_X_train, 'data/pi/dbc/df_ht_X_train_asp_dbc_20231217_20240303.pickle')
# utils.save_pickle(df_X_test, 'data/pi/dbc/df_ht_X_test_asp_dbc_20231217_20240303.pickle')
# utils.save_pickle(df_y_train, 'data/pi/dbc/df_ht_y_train_asp_dbc_20231217_20240303.pickle')
# utils.save_pickle(df_y_test, 'data/pi/dbc/df_ht_y_test_asp_dbc_20231217_20240303.pickle')

utils.save_pickle(df_X_train, 'data/pi/dbc/asp1/df_ht_X_train_asp_dbc_asp1_20231217_20240303.pickle')
utils.save_pickle(df_X_test, 'data/pi/dbc/asp1/df_ht_X_test_asp_dbc_asp1_20231217_20240303.pickle')
utils.save_pickle(df_y_train, 'data/pi/dbc/asp1/df_ht_y_train_asp_dbc_asp1_20231217_20240303.pickle')
utils.save_pickle(df_y_test, 'data/pi/dbc/asp1/df_ht_y_test_asp_dbc_asp1_20231217_20240303.pickle')

In [ ]:
# df_X_train = utils.load_pickle('data/pi/df_ht_X_train_fusion_20231217_20240204.pickle')
# df_y_train = utils.load_pickle('data/pi/df_ht_y_train_fusion_20231217_20240204.pickle')

# df_X_train = utils.load_pickle('data/pi/df_ht_X_train_asp_bcd_20231217_20240204.pickle')
# df_y_train = utils.load_pickle('data/pi/df_ht_y_train_asp_bcd_20231217_20240204.pickle')

# df_X_train = utils.load_pickle('data/pi/dbc/df_ht_X_train_asp_dbc_20231217_20240303.pickle')
# df_y_train = utils.load_pickle('data/pi/dbc/df_ht_y_train_asp_dbc_20231217_20240303.pickle')

df_X_train = utils.load_pickle('data/pi/dbc/asp1/df_ht_X_train_asp_dbc_asp1_20231217_20240303.pickle')
df_y_train = utils.load_pickle('data/pi/dbc/asp1/df_ht_y_train_asp_dbc_asp1_20231217_20240303.pickle')

print(df_X_train.shape)
print(df_y_train.shape)

In [ ]:
%%time
estimator_pi=lgb.LGBMClassifier(importance_type='gain')
estimator_pi.fit(df_X_train, df_y_train)

# utils.save_pickle(estimator_pi, 'data/pi/estimator_pi_ht_fusion_20231217_20240204.pickle')
# utils.save_pickle(estimator_pi, 'data/pi/estimator_pi_ht_asp_bcd_20231217_20240204.pickle')
# utils.save_pickle(estimator_pi, 'data/pi/dbc/estimator_pi_ht_asp_dbc_20231217_20240303.pickle')
utils.save_pickle(estimator_pi, 'data/pi/dbc/asp1/estimator_pi_ht_asp_dbc_asp1_20231217_20240303.pickle')

In [ ]:
# df_X_test = utils.load_pickle('data/pi/df_ht_X_test_fusion_20231217_20240204.pickle')
# df_y_test = utils.load_pickle('data/pi/df_ht_y_test_fusion_20231217_20240204.pickle')

# df_X_test = utils.load_pickle('data/pi/df_ht_X_test_asp_bcd_20231217_20240204.pickle')
# df_y_test = utils.load_pickle('data/pi/df_ht_y_test_asp_bcd_20231217_20240204.pickle')

# df_X_test = utils.load_pickle('data/pi/dbc/df_ht_X_test_asp_dbc_20231217_20240303.pickle')
# df_y_test = utils.load_pickle('data/pi/dbc/df_ht_y_test_asp_dbc_20231217_20240303.pickle')

df_X_test = utils.load_pickle('data/pi/dbc/asp1/df_ht_X_test_asp_dbc_asp1_20231217_20240303.pickle')
df_y_test = utils.load_pickle('data/pi/dbc/asp1/df_ht_y_test_asp_dbc_asp1_20231217_20240303.pickle')

print(df_X_test.shape)
print(df_y_test.shape)

In [ ]:
# estimator_pi = utils.load_pickle('data/pi/estimator_pi_ht_fusion_20231217_20240204.pickle')
# estimator_pi = utils.load_pickle('data/pi/estimator_pi_ht_asp_bcd_20231217_20240204.pickle')

# estimator_pi = utils.load_pickle('data/pi/dbc/estimator_pi_ht_asp_dbc_20231217_20240303.pickle')

estimator_pi = utils.load_pickle('data/pi/dbc/asp1/estimator_pi_ht_asp_dbc_asp1_20231217_20240303.pickle')

estimator_pi

In [ ]:
%%time
pi = permutation_importance(estimator=estimator_pi, X=df_X_test, y=df_y_test, n_jobs=8)

# utils.save_pickle(pi, 'data/pi/pi_ht_fusion_20231217_20240204.pickle')
# utils.save_pickle(pi, 'data/pi/pi_ht_asp_bcd_20231217_20240204.pickle')

# utils.save_pickle(pi, 'data/pi/dbc/pi_ht_asp_dbc_20231217_20240303.pickle')

utils.save_pickle(pi, 'data/pi/dbc/asp1/pi_ht_asp_dbc_asp1_20231217_20240303.pickle')

In [ ]:
# list_feats_x_ht = utils.load_pickle('data/list_feats/list_feats_x_ht_fusion_20231217_20240204.pickle')

# list_feats_x_ht_1 = utils.load_pickle('data/list_feats/list_feats_x_ht_asp1_20231217_20240204.pickle')
list_feats_x_ht_2 = utils.load_pickle('data/list_feats/list_feats_x_ht_asp2_20231217_20240204.pickle')
# list_feats_x_ht_3 = utils.load_pickle('data/list_feats/list_feats_x_ht_bcd_20231217_20240204.pickle')

list_feats_x_ht_3 = utils.load_pickle('data/list_feats/dbc/list_feats_x_ht_dbc_20231217_20240303.pickle')

list_feats_x_ht_1 = [
    x for x in utils.load_pickle('data/list_feats/list_feats_x_ht_asp1_20231217_20240204.pickle')
    if x not in list(pd.read_csv('data/pi/dbc/asp1/cm_credit_card.txt', sep='\t', encoding='utf-8')['feats'])
]

list_feats_x_ht = list_feats_x_ht_1 + list_feats_x_ht_2 + list_feats_x_ht_3

print(len(list_feats_x_ht_1))
print(len(list_feats_x_ht_2))
print(len(list_feats_x_ht_3))

print(len(list_feats_x_ht))

In [ ]:
df_pi = pd.DataFrame({'feature': list_feats_x_ht, 'permutation_importance_mean': pi.importances_mean})
print(df_pi.shape)
df_pi.head()

In [ ]:
df_pi.sort_values(by=['permutation_importance_mean'], ascending=[False], inplace=True)
df_pi

In [ ]:
df_pi[['permutation_importance_mean']].quantile([x/10 for x in range(11)]).T

In [ ]:
list_feats_x_pi_no_select = df_pi[(df_pi['permutation_importance_mean']<=0.0)]['feature'].values.tolist()
print(len(list_feats_x_pi_no_select))
list_feats_x_pi_no_select[:10]

In [ ]:
# utils.save_pickle(list_feats_x_pi_no_select, 'data/list_feats/list_feats_x_pi_no_select_fusion_20231217_20240204.pickle')
# utils.save_pickle(list_feats_x_pi_no_select, 'data/list_feats/list_feats_x_pi_no_select_asp_bcd_20231217_20240204.pickle')

# utils.save_pickle(list_feats_x_pi_no_select, 'data/list_feats/dbc/list_feats_x_pi_no_select_asp_dbc_20231217_20240303.pickle')

utils.save_pickle(list_feats_x_pi_no_select, 'data/list_feats/dbc/asp1/list_feats_x_pi_no_select_asp_dbc_asp1_20231217_20240303.pickle')

In [ ]:
list_feats_x_pi = [x for x in list_feats_x_ht if x not in list_feats_x_pi_no_select]
print(len(list_feats_x_pi))
list_feats_x_pi[:10]

In [ ]:
[x for x in list_feats_x_pi if x in ['is_bind_credit_card_xxpay']]

In [ ]:
# utils.save_pickle(list_feats_x_pi, 'data/list_feats/list_feats_x_pi_fusion_20231217_20240204.pickle')
# utils.save_pickle(list_feats_x_pi, 'data/list_feats/list_feats_x_pi_asp_bcd_20231217_20240204.pickle')

# utils.save_pickle(list_feats_x_pi, 'data/list_feats/dbc/list_feats_x_pi_asp_dbc_20231217_20240303.pickle')

utils.save_pickle(list_feats_x_pi, 'data/list_feats/dbc/asp1/list_feats_x_pi_asp_dbc_asp1_20231217_20240303.pickle')

## 全量数据拉取sql处理

In [ ]:
def del_sql_coalesce(sql_file_path, special_words, sql_file_path_new):
    with open(sql_file_path, 'r') as fi:
        list_sql = fi.readlines()
        
        str_pattern = r'|'.join(['( '+x+', )' for x in special_words])
        str_pattern_end = r'|'.join(['( '+x+' )' for x in special_words])
        str_pattern_from = r'^from '
        pattern = re.compile(str_pattern)
        pattern_end = re.compile(str_pattern_end)
        pattern_from = re.compile(str_pattern_from)
        
        list_sql_valid = []
        list_sql_valid_front = []
        list_sql_valid_back = []
        
        index_front = 0
        for i in range(len(list_sql)):
            if re.search(pattern_from, list_sql[i]):
                index_front = i
                break
        list_sql_valid_back = list_sql[index_front+1:]
        
        try:
            with tqdm(list(range(len(list_sql[:index_front])))) as t:
                for i in t:
                    if i == index_front - 1:
                        if not re.search(str_pattern_end, list_sql[i]):
                            list_sql_valid_front.append(list_sql[i])
                    else:
                        if not re.search(pattern, list_sql[i]):
                            list_sql_valid_front.append(list_sql[i])
        except KeyboardInterrupt:
            t.close()
            raise
        t.close()
        list_sql_valid = list_sql_valid_front + [list_sql[index_front]] + list_sql_valid_back
        
        with open(sql_file_path_new, 'w') as fo:
            fo.writelines(list_sql_valid)
            

def del_sql_select_origin(sql_file_path, special_words, sql_file_path_new):
    with open(sql_file_path, 'r') as fi:
        list_sql = fi.readlines()
        
        str_pattern_1 = r'|'.join(['(\t'+x+', )' for x in special_words])
        str_pattern_2 = r'|'.join(['(    '+x+', )' for x in special_words])
        str_pattern_3 = r'|'.join(['(\t'+x+' )' for x in special_words])
        str_pattern_4 = r'|'.join(['(    '+x+' )' for x in special_words])
        str_pattern_from = r'^from '
        pattern_1 = re.compile(str_pattern_1)
        pattern_2 = re.compile(str_pattern_2)
        pattern_3 = re.compile(str_pattern_3)
        pattern_4 = re.compile(str_pattern_4)
        pattern_from = re.compile(str_pattern_from)
        
        list_sql_valid = []
        list_sql_valid_front = []
        list_sql_valid_back = []
        
        index_front = 0
        for i in range(len(list_sql)):
            if re.search(pattern_from, list_sql[i]):
                index_front = i
                break
        list_sql_valid_front = list_sql[:index_front]
        
        try:
            with tqdm(list(range(len(list_sql[index_front+1:])))) as t:
                for i in t:
                    if not re.search(pattern_1, list_sql[index_front+1+i]) \
                        and not re.search(pattern_2, list_sql[index_front+1+i]) \
                        and not re.search(pattern_3, list_sql[index_front+1+i]) \
                        and not re.search(pattern_4, list_sql[index_front+1+i]):
                        list_sql_valid_back.append(list_sql[index_front+1+i])
        except KeyboardInterrupt:
            t.close()
            raise
        t.close()
        list_sql_valid = list_sql_valid_front + [list_sql[index_front]] + list_sql_valid_back
        
        with open(sql_file_path_new, 'w') as fo:
            fo.writelines(list_sql_valid)
            
            
def correct_end(sql_file_path, sql_file_path_new):
    with open(sql_file_path, 'r') as fi:
        list_sql = fi.readlines()
        
        str_pattern = r'from '
        pattern = re.compile(str_pattern)
        
        try:
            with tqdm(list(range(len(list_sql)))) as t:
                for i in t:
                    if re.search(pattern, list_sql[i]):
                        if list_sql[i-1][-3:-1] == ', ':
                            list_sql[i-1] = list_sql[i-1][:-3] + list_sql[i-1][-2:]
        except KeyboardInterrupt:
            t.close()
            raise
        t.close()
        
        with open(sql_file_path_new, 'w') as fo:
            fo.writelines(list_sql)

In [ ]:
# list_feats_x = utils.load_pickle('data/list_feats/list_feats_x_pi_fusion_20231217_20240204.pickle')
# list_feats_x = utils.load_pickle('data/list_feats/list_feats_x_pi_asp_bcd_20231217_20240204.pickle')

# list_feats_x = utils.load_pickle('data/list_feats/dbc/list_feats_x_pi_asp_dbc_20231217_20240303.pickle')

list_feats_x = utils.load_pickle('data/list_feats/dbc/asp1/list_feats_x_pi_asp_dbc_asp1_20231217_20240303.pickle')

print(len(list_feats_x))
list_feats_x[:10]

In [ ]:
# list_feats_x_all = utils.load_pickle('data/list_feats/list_feats_x_fusion_20231217_20240204.pickle')
# list_feats_x_all = utils.load_pickle('data/list_feats/list_feats_x_asp1_20231217_20240204.pickle')
# list_feats_x_all = utils.load_pickle('data/list_feats/list_feats_x_asp2_20231217_20240204.pickle')
# list_feats_x_all = utils.load_pickle('data/list_feats/list_feats_x_bcd_20231217_20240204.pickle')

list_feats_x_all = utils.load_pickle('data/list_feats/dbc/list_feats_x_dbc_20231217_20240303.pickle')

print(len(list_feats_x_all))
list_feats_x_all[:10]

In [ ]:
list_feats_x_pi = [x for x in list_feats_x_all if x in list_feats_x]
print(len(list_feats_x_pi))

# utils.save_pickle(list_feats_x_pi, 'data/list_feats/list_feats_x_pi_asp1_20231217_20240204.pickle')
# utils.save_pickle(list_feats_x_pi, 'data/list_feats/list_feats_x_pi_asp2_20231217_20240204.pickle')
# utils.save_pickle(list_feats_x_pi, 'data/list_feats/list_feats_x_pi_bcd_20231217_20240204.pickle')

# utils.save_pickle(list_feats_x_pi, 'data/list_feats/dbc/list_feats_x_pi_asp1_20231217_20240303.pickle')
# utils.save_pickle(list_feats_x_pi, 'data/list_feats/dbc/list_feats_x_pi_asp2_20231217_20240303.pickle')
# utils.save_pickle(list_feats_x_pi, 'data/list_feats/dbc/list_feats_x_pi_dbc_20231217_20240303.pickle')

# utils.save_pickle(list_feats_x_pi, 'data/list_feats/dbc/asp1/list_feats_x_pi_asp1_20231217_20240303.pickle')
# utils.save_pickle(list_feats_x_pi, 'data/list_feats/dbc/asp1/list_feats_x_pi_asp2_20231217_20240303.pickle')
utils.save_pickle(list_feats_x_pi, 'data/list_feats/dbc/asp1/list_feats_x_pi_dbc_20231217_20240303.pickle')

In [ ]:
list_feats_x_del = [x for x in list_feats_x_all if x not in list_feats_x_pi]
list_feats_x_del = ['asd'] if len(list_feats_x_del) == 0 else list_feats_x_del
print(len(list_feats_x_del))
list_feats_x_del[:10]

In [ ]:
special_words = list_feats_x_del

# sql_file_path = 'sample_label_feature_fusion.sql'
# sql_file_path_new = 'sample_label_feature_fusion_.sql'

# sql_file_path = 'sample_label_feature_asp1.sql'
# sql_file_path_new = 'sample_label_feature_asp1_.sql'
# sql_file_path_new = 'sample_label_feature_asp1_1.sql'
# sql_file_path_new = 'sample_label_feature_asp1_2.sql'

# sql_file_path = 'sample_label_feature_asp2.sql'
# sql_file_path_new = 'sample_label_feature_asp2_.sql'
# sql_file_path_new = 'sample_label_feature_asp2_1.sql'
# sql_file_path_new = 'sample_label_feature_asp2_2.sql'

# sql_file_path = 'sample_label_feature_bank_card_debit.sql'
# sql_file_path_new = 'sample_label_feature_bank_card_debit_.sql'

sql_file_path = 'sample_label_feature_xxpay_bank_card.sql'
# sql_file_path_new = 'sample_label_feature_xxpay_bank_card_.sql'
sql_file_path_new = 'sample_label_feature_xxpay_bank_card_1.sql'

del_sql_coalesce(sql_file_path, special_words, sql_file_path_new)
del_sql_select_origin(sql_file_path_new, special_words, sql_file_path_new)
correct_end(sql_file_path_new, sql_file_path_new)

In [ ]:
def del_sql_coalesce_table(sql_file_path, special_words, sql_file_path_new):
    with open(sql_file_path, 'r') as fi:
        list_sql = fi.readlines()
        
        str_pattern_end = r'|'.join(['( `'+x+'` )' for x in special_words])
        str_pattern_from = r'^from '
        pattern = re.compile(str_pattern)
        pattern_end = re.compile(str_pattern_end)
        pattern_from = re.compile(str_pattern_from)
        
        list_sql_valid = []
        list_sql_valid_front = []
        list_sql_valid_back = []
        
        index_front = 0
        for i in range(len(list_sql)):
            if re.search(pattern_from, list_sql[i]):
                index_front = i
                break
        list_sql_valid_back = list_sql[index_front+1:]
        
        try:
            with tqdm(list(range(len(list_sql[:index_front])))) as t:
                for i in t:
                    if not re.search(str_pattern_end, list_sql[i]):
                        list_sql_valid_front.append(list_sql[i])
        except KeyboardInterrupt:
            t.close()
            raise
        t.close()
        list_sql_valid = list_sql_valid_front + [list_sql[index_front]] + list_sql_valid_back
        
        with open(sql_file_path_new, 'w') as fo:
            fo.writelines(list_sql_valid)